### Define Mappings

In [1]:
# Create mapping for letters a-z (0-25)
char_to_num = {chr(i + 97): i for i in range(26)}

# Add special mappings
char_to_num.update({
    'del': 26,
    'nothing': 27,
    'space': 28
})

# Create reverse mapping
num_to_char = {v: k for k, v in char_to_num.items()}

# Print mappings to verify
print("Character to Number mapping:")
print(char_to_num)
print("\nNumber to Character mapping:")
print(num_to_char)

Character to Number mapping:
{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, 'del': 26, 'nothing': 27, 'space': 28}

Number to Character mapping:
{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z', 26: 'del', 27: 'nothing', 28: 'space'}


In [2]:
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np

In [3]:
data_dict = pickle.load(open('data.pkl', 'rb'))

data = np.asarray(data_dict['data'])
labels = np.asarray(data_dict['labels'])
print(f'Data shape: {data.shape}')
print(f'Labels shape: {labels.shape}')

Data shape: (2201, 42)
Labels shape: (2201, 1)
